#### Tuesday, November 5, 2024

mamba activate langchain-chroma

[Build a Local RAG Application](https://python.langchain.com/docs/tutorials/local_rag/)

Gonna try using LMStudio and HuggingFace embeddings.

Nice! This all runs in one pass.

In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
# model_kwargs = {'device': 'cpu'}
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hfEmbeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/home/rob/miniforge3/envs/langchain-chroma/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

# local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

# vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)
vectorstore = Chroma.from_documents(documents=all_splits, embedding=hfEmbeddings)

In [4]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [5]:
docs[0]

Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}, page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.')

In [6]:
# from langchain_ollama import ChatOllama

# model = ChatOllama(
#     # model="llama3.1:8b",
#        model="llama3.1:latest",
#        #model="llama3.2:latest",
# )

from langchain_openai import ChatOpenAI

model = ChatOpenAI(base_url="http://localhost:1234/v1", 
                   # model = "hermes-3-llama-3.1-8b",  # do not pass in an unrecognized model name ... 
                   api_key="lm-studio", 
                   temperature=0)

In [7]:
response_message = model.invoke(
    "Simulate a rap battle between Stephen Colbert and John Oliver"
)

print(response_message.content)

*Stephen Colbert steps up to the mic, grinning mischievously*

Yo John, what's up? Stephen Colbert in the house, 
You think you're funny, but your jokes are a mouse,
Of course you've got that British charm,
But I'm here to show you how this game is played from the start!

*John Oliver takes his turn, smirking confidently*

Stephen, my man, you've got some skills,
But I've been around the block and know how to kill,
Your humor's clever, but it's all a bit too precious,
I'll take that microphone and show you what real is!

*Colbert comes back even stronger, not missing a beat*

Oh John, you think you can best me?
You're just another talking head, as far as I see,
But I'll give you this - your wit's sharp like a knife,
Now let's see if you can keep up with my rhymes and life!

*Oliver grins, clearly enjoying the back-and-forth*

Stephen, you're quick, but I'm quicker,
I've got that British bite that will make you shudder,
You may have Late Night, but I've got Last Week Tonight,
And I'll u

#### Using in a chain

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

chain.invoke(docs)

'The main themes in these retrieved documents are:\n\n1. Task decomposition methods:\n   - Using simple prompting like "Steps for XYZ.\\n1."\n   - Asking "What are the subgoals for achieving XYZ?"\n   - Employing task-specific instructions, such as "Write a story outline." for writing a novel\n   - Incorporating human inputs\n\n2. Challenges in long-term planning and task decomposition:\n   - Planning over a lengthy history is difficult\n   - Effectively exploring the solution space remains challenging\n   - LLMs struggle to adjust plans when faced with unexpected errors, making them less robust compared to humans who learn from trial and error\n\n3. Planning process:\n   - Breaking down large tasks into smaller, manageable subgoals for efficient handling of complex tasks\n   - Engaging in self-criticism and self-reflection over past actions\n   - Learning from mistakes and refining them for future steps to improve the quality of final results\n\n4. Memory and task execution:\n   - Exp

#### Q&A

In [9]:
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

# Run
chain.invoke({"context": docs, "question": question})

'Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'

#### Q&A with retrieval

In [10]:
retriever = vectorstore.as_retriever()

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

In [11]:
question = "What are the approaches to Task Decomposition?"

qa_chain.invoke(question)

'Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'